## Assignment for Module 6

## Author: Peihang Xu

In this assignment you will continue working with the housing price per district from the previous module assignment, this time training SVM models, both for regression and classification.

#### Getting the data for the assignment (similar to the notebook from chapter 2 of Hands-On...)

In [1]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [2]:
fetch_housing_data()

In [3]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

In [4]:
housing = load_housing_data()

### Fix the categories in the categorical variable

In [5]:
d = {'<1H OCEAN':'LESS_1H_OCEAN', 'INLAND':'INLAND', 'ISLAND':'ISLAND', 'NEAR BAY':'NEAR_BAY', 'NEAR OCEAN':'NEAR_OCEAN'}
housing['ocean_proximity'] = housing['ocean_proximity'].map(lambda s: d[s])

### Add 2 more features

In [6]:
housing["rooms_per_household"] = housing["total_rooms"]/housing["households"]
housing["population_per_household"]=housing["population"]/housing["households"]

### Fix missing data

In [7]:
median = housing["total_bedrooms"].median()
housing["total_bedrooms"].fillna(median, inplace=True) 

### Create dummy variables based on the categorical variable

In [8]:
one_hot = pd.get_dummies(housing['ocean_proximity'])
housing = housing.drop('ocean_proximity', axis=1)
housing = housing.join(one_hot)

### Check the data

In [9]:
housing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 16 columns):
longitude                   20640 non-null float64
latitude                    20640 non-null float64
housing_median_age          20640 non-null float64
total_rooms                 20640 non-null float64
total_bedrooms              20640 non-null float64
population                  20640 non-null float64
households                  20640 non-null float64
median_income               20640 non-null float64
median_house_value          20640 non-null float64
rooms_per_household         20640 non-null float64
population_per_household    20640 non-null float64
INLAND                      20640 non-null uint8
ISLAND                      20640 non-null uint8
LESS_1H_OCEAN               20640 non-null uint8
NEAR_BAY                    20640 non-null uint8
NEAR_OCEAN                  20640 non-null uint8
dtypes: float64(11), uint8(5)
memory usage: 1.8 MB


In [10]:
housing.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,rooms_per_household,population_per_household,INLAND,ISLAND,LESS_1H_OCEAN,NEAR_BAY,NEAR_OCEAN
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,452600.0,6.984127,2.555556,0,0,0,1,0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,358500.0,6.238137,2.109842,0,0,0,1,0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,352100.0,8.288136,2.802260,0,0,0,1,0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,341300.0,5.817352,2.547945,0,0,0,1,0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,342200.0,6.281853,2.181467,0,0,0,1,0


### Partition into train and test

Use train_test_split from sklearn.model_selection to partition the dataset into 70% for training and 30% for testing.

You can use the 70% for training set as both training and validation by using cross-validation.


In [11]:
from sklearn.model_selection import train_test_split
train_set, test_set = train_test_split(housing, test_size=0.3, random_state=42)

### Features

In [12]:
target = 'median_house_value'
features = list(train_set.columns)
features = [f for f in features if f!=target]

In [13]:
X_tr = train_set[features]
y_tr = train_set[[target]]

X_te = test_set[features]
y_te = test_set[[target]]

### Scaling features

Similarly, use StandardScaler from sklearn.preprocessing to normalize the training and testing data, using the training data

In [14]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_tr)
X_tr = scaler.transform(X_tr)
X_te = scaler.transform(X_te)

#### Comparing models

In [15]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import numpy as np

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())

### Linear regression on original features (no transformations) --- benchmark

In [16]:
from sklearn.linear_model import LinearRegression
lin_scores = cross_val_score(LinearRegression(), train_set[features], train_set[target], scoring="neg_mean_squared_error", cv=4)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

Scores: [70142.55721218 67456.39127204 67318.3258893  70866.26065275]
Mean: 68945.88375656866


# 1. Support Vector Machines for Regression

#### (a) In this exercise your goal is to tune SVR with FBR kernel, and make the average score mean_squared_error over 3-folds (cv=3) below 58000. 

You are encouraged to try optimizing any of the hyper-parameters of SVR

See http://scikit-learn.org/stable/modules/generated/sklearn.svm.SVR.html for more details

However, as a hint, you can focus on C and gamma. 

Hint 2: if when you try different values for a hyper-parameter, the optimal models corresponds to one of the extreme values in your range, that probably means you can keep improving your solution by considering values beyond the current range.



In [17]:
from sklearn.svm import SVR

C_vals = [0.1,10,10000,100000] ## YOUR VALUES FOR C ##
gamma_vals = [0.1,1,100,1000] ## YOUR VALUES FOR gamma ## 

param_grid = [{'C':C_vals, 'gamma':gamma_vals}]
grid_search_rbf = GridSearchCV(SVR(kernel='rbf'), param_grid, cv=3,scoring='neg_mean_squared_error')
grid_search_rbf.fit(X_tr, np.ravel(y_tr))

GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=SVR(C=1.0, cache_size=200, coef0=0.0, degree=3,
                           epsilon=0.1, gamma='auto_deprecated', kernel='rbf',
                           max_iter=-1, shrinking=True, tol=0.001,
                           verbose=False),
             iid='warn', n_jobs=None,
             param_grid=[{'C': [0.1, 10, 10000, 100000],
                          'gamma': [0.1, 1, 100, 1000]}],
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='neg_mean_squared_error', verbose=0)

In [18]:
print(grid_search_rbf.best_params_)
print("Best score:",np.sqrt(-grid_search_rbf.best_score_))

{'C': 100000, 'gamma': 0.1}
Best score: 57255.36016378938


In [19]:
cvres_rbf = grid_search_rbf.cv_results_
cvres_rbf["mean_test_score"]
cvres_rbf["params"]
for mean_score, params in zip(cvres_rbf["mean_test_score"], cvres_rbf["params"]):
    print(np.sqrt(-mean_score), params)
print("Best_parameters:", grid_search_rbf.best_params_)

118994.53665534152 {'C': 0.1, 'gamma': 0.1}
119015.42561724437 {'C': 0.1, 'gamma': 1}
119017.78748915922 {'C': 0.1, 'gamma': 100}
119017.78786776128 {'C': 0.1, 'gamma': 1000}
116723.1737082353 {'C': 10, 'gamma': 0.1}
118706.78465088698 {'C': 10, 'gamma': 1}
119017.74998704392 {'C': 10, 'gamma': 100}
119017.78784663115 {'C': 10, 'gamma': 1000}
62439.06054204476 {'C': 10000, 'gamma': 0.1}
77804.73236805746 {'C': 10000, 'gamma': 1}
118996.50738233619 {'C': 10000, 'gamma': 100}
119028.09027110973 {'C': 10000, 'gamma': 1000}
57255.36016378938 {'C': 100000, 'gamma': 0.1}
61010.1039625056 {'C': 100000, 'gamma': 1}
117182.9081831695 {'C': 100000, 'gamma': 100}
117437.63971796147 {'C': 100000, 'gamma': 1000}
Best_parameters: {'C': 100000, 'gamma': 0.1}


### Performance on Test Set

In [20]:
from sklearn.metrics import mean_squared_error

final_model = grid_search_rbf.best_estimator_   ## THIS SHOULD BE THE BEST GRID_SEARCH ##

y_te_estimation = final_model.predict(X_te)
y_tr_estimation = final_model.predict(X_tr)

final_mse = mean_squared_error(y_te, y_te_estimation)
final_rmse = np.sqrt(final_mse)
print("RMSE for test set:",final_rmse)
final_mse_tr = mean_squared_error(y_tr, y_tr_estimation)
final_rmse_tr = np.sqrt(final_mse_tr)
print("RMSE for train set:",final_rmse_tr)

RMSE for test set: 56466.15041953528
RMSE for train set: 54691.358731568034


#### As expected, RMSE for train is slightly higher for test set

# 2. SVM for Classification

Now we transform the continuous target into a binary variable, indicating whether or not the price is above the average $179700


In [21]:
from sklearn.metrics import accuracy_score
# Filter convergence warnings
from sklearn.exceptions import ConvergenceWarning
import warnings
warnings.filterwarnings('ignore', category=ConvergenceWarning)


In [22]:
np.median(housing[['median_house_value']])

179700.0

#### Binary target variable

In [23]:
y_tr_b = 1*np.ravel(y_tr>=179700.0)
y_te_b = 1*np.ravel(y_te>=179700.0)

#### Linear SVM for classification

In [24]:
from sklearn.svm import LinearSVC

In [25]:
lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_tr, y_tr_b)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
          verbose=0)

In [26]:
y_pred = lin_clf.predict(X_tr)
print('Accuracy score for training set:',accuracy_score(y_tr_b, y_pred))

Accuracy score for training set: 0.8385935769656699


In [27]:
y_pred_te = lin_clf.predict(X_te)
print('Accuracy score for test set:',accuracy_score(y_te_b, y_pred_te))

Accuracy score for test set: 0.8389857881136951


In [28]:
from sklearn.metrics import roc_auc_score
# roc_auc_score expects 
roc_auc_score(y_tr_b, y_pred)

0.8386166219228823

In [29]:
from sklearn.model_selection import cross_val_score

Linear_SVC_scores = cross_val_score(lin_clf, X_tr, y_tr_b,
                         scoring="accuracy", cv=3)

print('Crosss validation scores:',Linear_SVC_scores.mean())

Crosss validation scores: 0.8374859514075578


### (a) Does SVC (with default hyper-parameters) improve the performance of the linear SVM?

In [30]:
from sklearn.svm import SVC

In [31]:
SVC_clf = SVC()
SVC_clf.fit(X_tr, y_tr_b)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [32]:
y_pred_SVC = SVC_clf.predict(X_tr)
accuracy_score(y_tr_b, y_pred_SVC)

0.866140642303433

In [33]:
SVC_scores = cross_val_score(SVC_clf, X_tr, y_tr_b,
                         scoring="accuracy", cv=3)

print('Crosss validation scores:',SVC_scores.mean())

/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


Crosss validation scores: 0.8590802051873722


In [34]:
y_pred = SVC_clf.predict(X_tr)
print('Accuracy score for training set:',accuracy_score(y_tr_b, y_pred))
y_pred_te = SVC_clf.predict(X_te)
print('Accuracy score for test set:',accuracy_score(y_te_b, y_pred_te))

Accuracy score for training set: 0.866140642303433
Accuracy score for test set: 0.8624031007751938


#### Accuracy increases a little bit comparing to Linear SVM

### (b) Use randomized search to tune hyper-parameters of SVC and improve its performance

In [35]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

In [36]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'C': randint(low=1, high=200),
    }

SVC_clf_rd = SVC(kernel = 'rbf',gamma = 'auto')
rnd_search = RandomizedSearchCV(SVC_clf_rd, param_distribs,
                                n_iter=5, cv=3, scoring='accuracy', random_state=42)
rnd_search.fit(X_tr, y_tr_b)

RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=SVC(C=1.0, cache_size=200, class_weight=None,
                                 coef0=0.0, decision_function_shape='ovr',
                                 degree=3, gamma='auto', kernel='rbf',
                                 max_iter=-1, probability=False,
                                 random_state=None, shrinking=True, tol=0.001,
                                 verbose=False),
                   iid='warn', n_iter=5, n_jobs=None,
                   param_distributions={'C': <scipy.stats._distn_infrastructure.rv_frozen object at 0x10d304bd0>},
                   pre_dispatch='2*n_jobs', random_state=42, refit=True,
                   return_train_score=False, scoring='accuracy', verbose=0)

In [37]:
cvres_SVC = rnd_search.cv_results_
cvres_SVC["mean_test_score"]
cvres_SVC["params"]
for mean_score, params in zip(cvres_SVC["mean_test_score"], cvres_SVC["params"]):
    print(mean_score, params)
print("Best_parameters:", rnd_search.best_params_)
print(rnd_search.best_params_)
print("Best cross validation score:",rnd_search.best_score_)

0.874031007751938 {'C': 103}
0.873062015503876 {'C': 180}
0.8737541528239202 {'C': 93}
0.8695321151716501 {'C': 15}
0.8741002214839424 {'C': 107}
Best_parameters: {'C': 107}
{'C': 107}
Best cross validation score: 0.8741002214839424


In [38]:
final_model_SVC = rnd_search.best_estimator_

y_pred = final_model_SVC.predict(X_tr)
print('Accuracy score for training set:',accuracy_score(y_tr_b, y_pred))
y_pred_te = final_model_SVC.predict(X_te)
print('Accuracy score for test set:',accuracy_score(y_te_b, y_pred_te))

Accuracy score for training set: 0.9004014396456257
Accuracy score for test set: 0.8766149870801033


#### Performance improves as expected

### (c) Train a Logistic Regression (search teh best hyper-parameters) and compare its performance with SVC 

In [39]:
from sklearn.linear_model import LogisticRegression

lr_clf = LogisticRegression(class_weight = 'balanced')
lr_clf.fit(X_tr, y_tr_b)

# y_pred_lr = lr_clf.predict(X_tr)
# accuracy_score(y_tr_b, y_pred_lr)

param_distribs = {
        'C': [1,100,1000],
        'solver': ['lbfgs','sag','liblinear'],
        'max_iter':[100,1000,10000],
    }

lr_clf_gs = LogisticRegression(class_weight = 'balanced')
grid_search_lr = GridSearchCV(lr_clf_gs, param_distribs,
                             cv=3, scoring='accuracy')
grid_search_lr.fit(X_tr, y_tr_b)

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


GridSearchCV(cv=3, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight='balanced',
                                          dual=False, fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [1, 100, 1000], 'max_iter': [100, 1000, 10000],
                         'solver': ['lbfgs', 'sag', 'liblinear']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=0)

In [40]:
cvres_lr = grid_search_lr.cv_results_
cvres_lr["mean_test_score"]
cvres_lr["params"]
for mean_score, params in zip(cvres_lr["mean_test_score"], cvres_lr["params"]):
    print(mean_score, params)
print("Best_parameters:", grid_search_lr.best_params_)
print(grid_search_lr.best_params_)
print('Best cross validation score:',grid_search_lr.best_score_)

0.8376937984496124 {'C': 1, 'max_iter': 100, 'solver': 'lbfgs'}
0.8365863787375415 {'C': 1, 'max_iter': 100, 'solver': 'sag'}
0.8376937984496124 {'C': 1, 'max_iter': 100, 'solver': 'liblinear'}
0.8376937984496124 {'C': 1, 'max_iter': 1000, 'solver': 'lbfgs'}
0.8375553709856035 {'C': 1, 'max_iter': 1000, 'solver': 'sag'}
0.8376937984496124 {'C': 1, 'max_iter': 1000, 'solver': 'liblinear'}
0.8376937984496124 {'C': 1, 'max_iter': 10000, 'solver': 'lbfgs'}
0.8375553709856035 {'C': 1, 'max_iter': 10000, 'solver': 'sag'}
0.8376937984496124 {'C': 1, 'max_iter': 10000, 'solver': 'liblinear'}
0.8377630121816169 {'C': 100, 'max_iter': 100, 'solver': 'lbfgs'}
0.8366555924695459 {'C': 100, 'max_iter': 100, 'solver': 'sag'}
0.8378322259136213 {'C': 100, 'max_iter': 100, 'solver': 'liblinear'}
0.8377630121816169 {'C': 100, 'max_iter': 1000, 'solver': 'lbfgs'}
0.8377630121816169 {'C': 100, 'max_iter': 1000, 'solver': 'sag'}
0.8378322259136213 {'C': 100, 'max_iter': 1000, 'solver': 'liblinear'}
0.8377

In [41]:
final_model_lr = grid_search_lr.best_estimator_

y_pred = final_model_lr.predict(X_tr)
print('Accuracy score for training set:',accuracy_score(y_tr_b, y_pred))
y_pred_te = final_model_lr.predict(X_te)
print('Accuracy score for test set:',accuracy_score(y_te_b, y_pred_te))

Accuracy score for training set: 0.8387320044296789
Accuracy score for test set: 0.8409237726098191


#### The performance of logistic regression is worse than using SVMs